In [ ]:
# ================
# 1. IMPORTS
# ================
import os
import logging
import warnings
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from sklearn.model_selection import (
    train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, RandomizedSearchCV, RepeatedStratifiedKFold
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, 
    HistGradientBoostingClassifier
)
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Interpretability
import shap
from sklearn.inspection import permutation_importance, PartialDependenceDisplay

# Optimization
import optuna

In [ ]:
# ================
# 2. CONFIGURATION
# ================
RANDOM_STATE = 42
TEST_SIZE = 0.2
N_SPLITS_CV = 5
SCORING_METRIC = 'roc_auc'
VERBOSE = 1

CPU_COUNT = os.cpu_count()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Preprocessing

In [16]:
# ================
# 3. Import seven waves of data
# ================

# Specify the directory path
directory_path = '~/work/vaping_project_data/original_all_core'
# Expand the ~ to the full home directory path
directory_path = os.path.expanduser(directory_path)

# List all files ending with '0810.tsv' in the specified directory
files = [f for f in os.listdir(directory_path) if f.endswith('0810.tsv')]

# Create a dictionary to store individual dataframes
dataframes = {}

# Read each file into a separate dataframe
for file in files:
    file_path = os.path.join(directory_path, file)
    try:
        # Use the file name (without extension) as the key
        df_name = file.replace('.tsv', '')  # Remove .tsv from the filename
        # Read the file with low_memory=False to handle mixed types
        dataframes[df_name] = pd.read_csv(file_path, sep='\t', low_memory=False)
        print(f"Successfully read: {file} into dataframe '{df_name}'")
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Example: Accessing a specific dataframe
# If you want to access the dataframe for 'original_core_2017_0810', you can do:
# df_2017 = dataframes['original_core_2017_0810']

Successfully read: original_core_2017_0810.tsv into dataframe 'original_core_2017_0810'
Successfully read: original_core_2018_0810.tsv into dataframe 'original_core_2018_0810'
Successfully read: original_core_2019_0810.tsv into dataframe 'original_core_2019_0810'
Successfully read: original_core_2020_0810.tsv into dataframe 'original_core_2020_0810'
Successfully read: original_core_2021_0810.tsv into dataframe 'original_core_2021_0810'
Successfully read: original_core_2022_0810.tsv into dataframe 'original_core_2022_0810'
Successfully read: original_core_2023_0810.tsv into dataframe 'original_core_2023_0810'


In [ ]:
# ================
# 4. Basic Info of each dataset
# ================

# Loop through each dataframe in the dictionary
for df_name, df in dataframes.items():
    print(f"=== Basic Information for {df_name} ===")
    
    # Display the first few rows
    print("\nFirst 5 Rows:")
    print(df.head())
    
    # Display the last few rows
    print("\nLast 5 Rows:")
    print(df.tail())
    
    # Get dataset shape
    print(f"\nDataset Shape: {df.shape}")
    
    # Get column names
    print(f"\nColumn Names: {df.columns.tolist()}")
    
    # Get data types
    print(f"\nData Types:\n{df.dtypes}")
    
    # Check for missing values
    print(f"\nMissing Values:\n{df.isnull().sum()}")
    
    # Get summary statistics for numerical columns
    print(f"\nSummary Statistics:\n{df.describe(include='all')}")
    
    # Count unique values in each column
    print(f"\nUnique Values per Column:\n{df.nunique()}")
    
    # Check for duplicate rows
    print(f"\nNumber of Duplicate Rows: {df.duplicated().sum()}")
    
    # Print a separator for readability
    print("\n" + "=" * 50 + "\n")

In [19]:
# ================
# 5. Inner Join all 7 waves
# ================

# Step 1: Find common columns across all dataframes
common_columns = set(dataframes[next(iter(dataframes))].columns)  # Initialize with columns from the first dataframe
for df in dataframes.values():
    common_columns.intersection_update(df.columns)  # Keep only columns present in all dataframes

# Convert the set of common columns to a list
common_columns = list(common_columns)
print(f"Common Columns: {common_columns}")

# Step 2: Filter each dataframe to keep only the common columns
filtered_dataframes = {}
for df_name, df in dataframes.items():
    filtered_dataframes[df_name] = df[common_columns]
    print(f"Filtered {df_name} to keep common columns.")

# Step 3: Concatenate all filtered dataframes into a single dataframe
merged_df = pd.concat(filtered_dataframes.values(), ignore_index=True)

# Display basic info of the merged dataframe
# Get dataset shape
print(f"\nDataset Shape: {df.shape}")
    
# Get column names
print(f"\nColumn Names: {df.columns.tolist()}")
    
# Get data types
print(f"\nData Types:\n{df.dtypes}")
    
# Check for missing values
print(f"\nMissing Values:\n{df.isnull().sum()}")
    
# Get summary statistics for numerical columns
print(f"\nSummary Statistics:\n{df.describe(include='all')}")
    
# Count unique values in each column
print(f"\nUnique Values per Column:\n{df.nunique()}")
    
# Check for duplicate rows
print(f"\nNumber of Duplicate Rows: {df.duplicated().sum()}")
    
# Print a separator for readability
print("\n" + "=" * 50 + "\n")

Common Columns: ['V8530', 'V7313', 'V7128', 'V7670', 'V7243', 'V7470', 'V7612', 'V7380', 'V7451', 'V7176', 'V7719', 'V7564', 'V8538', 'V8513', 'V7464', 'V7227', 'V7301', 'V7359', 'V7320', 'V8462', 'V7659', 'V7480', 'V7251', 'V7452', 'V7448', 'V7134D', 'V7663', 'V7147', 'V7579', 'V7125', 'V7234', 'V7550', 'V7404', 'V7453', 'V8517', 'V7145', 'V7437', 'V507', 'V7433', 'V7471', 'V7357', 'V7116D', 'V7586', 'V8419', 'V7706', 'V7426', 'V7140', 'V7477', 'V7592', 'V7414', 'V7601', 'V7360', 'V7384', 'V7102D', 'V7446', 'V8418', 'V7214', 'V7669', 'V7224', 'V7668', 'V8501', 'V7488', 'V7119D', 'V7517', 'V7468', 'V7228', 'V7478', 'V8413', 'V7454', 'V8461', 'V7643', 'V7709', 'V7261', 'V7409', 'V7185', 'V7511', 'V7356', 'V7239', 'V8511', 'V8555', 'V7580', 'V7503', 'V7134', 'V7327', 'V1252', 'V7105', 'V7444', 'V7425', 'V8541', 'V7222', 'V7126', 'V7702', 'V7113D', 'V501', 'V7713', 'V7120D', 'V7254', 'V7494', 'V7124', 'V7661', 'V7146', 'V7344', 'V7427', 'V7717', 'V7429', 'V7541', 'V8447', 'V7516', 'V7111'